In [ ]:
! pip install fuzzywuzzy

In [2]:
import pandas as pd 
import numpy as np
from fuzzywuzzy import fuzz
from tqdm import tqdm
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
from collections import Counter

/Users/jerry/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
public = pd.read_csv('data_source/Financial_new.csv')
# public_rd = pd.read_csv('drive-download-20220516T014219Z-001/RD-expenses.csv')
patent_data = pd.read_csv('data_source/Final_Patent_Data.csv') 

FileNotFoundError: [Errno 2] No such file or directory: 'data_source/Financial_new.csv'

In [3]:
len(public.Company.unique())

27787

# Merge the public to public_rd

In [14]:
public.head(3)

,Unnamed: 0,Company,Year,Currency,Total Assets (millions),Employees (thousands),Company Status,Market Cap (millions),State Incorporated,S&P industry code,State,Public,Always Public
0,0,01 communique laboratory,2000,CAD,20.226,NaN,A,168.4491,NaN,185.0,ON,1,1
1,1,01 communique laboratory,2001,CAD,7.926,NaN,A,14.4000,NaN,185.0,ON,1,1
2,2,01 communique laboratory,2002,CAD,3.643,NaN,A,5.0954,NaN,185.0,ON,1,1


In [16]:
#drop rows with fyear contains NaNs
public_rd = public_rd[public_rd['fyear'].notna()]
public_rd.fyear = public_rd.fyear.apply(lambda x: int(x))

<ipython-input-16-936ad64f3a7c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  public_rd.fyear = public_rd.fyear.apply(lambda x: int(x))


In [18]:
public_info = public.merge(public_rd, how='left', left_on=['Company', 'Year'], 
                                right_on=['conm', 'fyear'])

In [19]:
public_info.head(3)

,Unnamed: 0,Company,Year,Currency,Total Assets (millions),Employees (thousands),Company Status,Market Cap (millions),State Incorporated,S&P industry code,...,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,xrd,costat
0,0,01 communique laboratory,2000,CAD,20.226,NaN,A,168.4491,NaN,185.0,...,2000.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,1.561,A
1,1,01 communique laboratory,2001,CAD,7.926,NaN,A,14.4000,NaN,185.0,...,2001.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,1.831,A
2,2,01 communique laboratory,2002,CAD,3.643,NaN,A,5.0954,NaN,185.0,...,2002.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,1.249,A


# Some string pre-processing and cleaning

In [5]:
def remove_chars_and_lower(string, chars, words):
    '''
    Function to remove quotation marks from a string, since there seems to be some issues of excess quotes
    placed into the company names
    '''
    string = ''.join(i for i in string if i not in chars)
    
    string_words = string.split()
    result_words  = [word for word in string_words if word.lower() not in words]
    result = ' '.join(result_words)
    
    return result.lower()

drop_chars = ['"', '.', '%']

drop_words = ['.com', 'llc', 'incorporated', 'inc', 'co', 'holdings', 'ho', 'corporation', 'corp', 
              'company', 'hldg', 'ent', 'ltd', 'association', 'foundation', 'club', 'fund', 'the', 'and', 'of', 
              'services', 'enterprises']

# companylist=['bottomline technologies inc', 'arrow international llc', 'american home products corp',
#             'aladdin knowledge systems ltd', "alan b clement"]
# # for i in companylist:
# #     print(re.sub(r"\W(inc|llc|corp|ltd|co|ltd)\b", "", i))
# comp_list = pd.DataFrame(companylist, columns = ['test'])   
# comp_list.test= comp_list.test.apply(lambda x: re.sub(r"\W(inc|llc|corp|ltd|co|ltd|lp)\b", "", x))

In [7]:
# trevor cleaning technique
public['Company'] = public['Company'].apply(lambda x: x.lower())
public['Company'] = public['Company'].apply(lambda x: remove_chars_and_lower(x, drop_chars, drop_words))

public_rd['conm'] = public_rd['conm'].apply(lambda x: x.lower())
public_rd['conm'] = public_rd['conm'].apply(lambda x: remove_chars_and_lower(x, drop_chars, drop_words))

In [8]:
# some low key data preprocesing

# lowercase patent_data
public.Company = public.Company.str.lower()
patent_data.correspondence_name = patent_data.correspondence_name.str.lower()
public_rd.conm = public_rd.conm.str.lower()

# remove all commas
public.Company = public.Company.apply(lambda x : x.replace(',',''))
patent_data.correspondence_name = patent_data.correspondence_name.apply(lambda x : x.replace(',',''))
public_rd.conm = public_rd.conm.apply(lambda x : x.replace(',',''))

# remove some specifics endings 
public.Company = public.Company.apply(lambda x: re.sub(r"\W(inc|llc|corp|ltd|co|ltd|lp)\b", "", x))
patent_data.correspondence_name = patent_data.correspondence_name.apply(lambda x: re.sub(r"\W(inc|llc|corp|ltd|co|ltd|lp)\b", "", x))
public_rd.conm = public_rd.conm.apply(lambda x: re.sub(r"\W(inc|llc|corp|ltd|co|ltd|lp)\b", "", x))


# stripping leading and trailing white spaces; and apply lower case to all names
public.Company = public.Company.replace(r"^ +| +$", r"", regex=True)
patent_data.correspondence_name = patent_data.correspondence_name.replace(r"^ +| +$", r"", regex=True)
public_rd.conm =  public_rd.conm.replace(r"^ +| +$", r"", regex=True)

Our goal: our goal is to create a mapping of company names. A mapping from each unique infringer to a patent comapny (if exits). 

Once we obtain the mapping, we can apply the mapping to our infringer dataset then we can do a brute force merging between infringer and patent dataset

In [9]:
unique_public = public.Company.unique()
unique_patent_co  = patent_data.correspondence_name.unique()
unique_rd_co = public_rd.conm.unique()

print("unique public co", len(unique_public))
print("unique patent owners",len(unique_patent_co))
print("unique r and d conm", len(unique_rd_co))

unique public co 27693
unique patent owners 364318
unique r and d conm 27717


# Now merge public and public r_d

# continue match

In [10]:
# return a match if there is a match that has a fuzzy score above min_score = 70
# return empty and -1 if there is no such match

def match_name(name, list_names, min_score=70):
    # -1 score incase we don't get any matches
    max_score = -1
    # Returning empty name for no match as well
    max_name = ""
    # Iternating over all names in the other
    for name2 in list_names:
        #Finding fuzzy match score
        score = fuzz.ratio(name, name2)
        # Checking if we are above our threshold and have a better score
        if (score > min_score) & (score > max_score):
            max_name = name2
            max_score = score
    return (max_name, max_score)


In [11]:
# test 1
match_name("tencent", unique_public, 70)

('tennant', 71)

# Prepartory work setting up the dataframe

In [12]:
your_part = unique_public # set it to your designated part

In [15]:
test_match = pd.DataFrame(your_part, columns = ["company_name"])
test_match["best_match"] = ""
test_match["best_score"] = 0 
test_match

,company_name,best_match,best_score
0,aar,,0
1,acf industries,,0
2,adc telecommunications,,0
3,afa protective systems,,0
4,afp imaging,,0
...,...,...,...
27688,contextlogic,,0
27689,immunitybio,,0
27690,karooooo,,0
27691,iveco group n v,,0


# Establish Sub-groups for Mapping

In [14]:
# we divide uniqnue patent company into subgroups by first two characters of the patent comapny name
unique_patent_co_df = pd.DataFrame(unique_patent_co, columns = ['patent_company'])
unique_patent_co_df['sort_gp'] = unique_patent_co_df['patent_company'].apply(lambda x: x[:2])
all_sort_gp = unique_patent_co_df['sort_gp'].unique()

print("all sub group")
for i in all_sort_gp:
  print(i)
print("total number of sub group start with first two characters" , len(all_sort_gp))


all sub group
br
co
dr
fr
p 
pa
sa
sh
& 
'm
(
( 
(2
(3
(4
(5
(7
(a
(b
(c
(d
(e
(g
(h
(i
(l
(m
(o
(r
(s
(t
(u
(y
) 
**

ar
in
mi
-
-a
-i
-o
-r
/
/ 
/a
/e
/g
/i
/m
/o
/r
/s
/u
0
00
08
0a
0b
0e
0f
0i
0l
0n
0p
0r
0s
1 
1-
10
11
12
13
14
15
16
17
18
19
1a
1c
1e
1f
1h
1l
1m
1n
1q
1r
1s
1w
2 
20
21
22
23
24
25
26
28
29
2f
2g
2k
2l
2n
2p
2s
2x
2y
3 
3-
30
31
32
33
34
36
37
38
39
3a
3b
3c
3d
3e
3h
3i
3m
3o
3r
3s
3w
3x
4 
4-
40
41
42
45
46
48
4c
4d
4f
4g
4i
4m
4p
5
50
51
53
54
55
57
58
5b
5f
5i
5o
5s
5t
60
61
62
63
65
67
6b
6d
6e
6i
6j
6k
6n
6o
6r
6u
7 
70
71
73
74
75
76
79
7a
7h
7o
7s
7u
80
81
82
85
86
87
8t
90
91
92
93
96
97
98
99
9c
9f
;a
<d
=n
??
?b
?i
?r
@@
@m
@v
[4
[i
[p
[x
\
\c
\m
]
]a
]o
]s
^
`
`a
`b
`e
`g
`h
`i
`l
`n
`o
`r
`t
`u
`w
a
a 
a&
a'
ag
a-
a/
a1
a7
a;
aa
ab
ac
ad
ae
af
ah
ai
aj
ak
al
am
an
ao
ap
aq
as
at
au
av
aw
ax
ay
az
b 
b&
b-
b2
b3
b;
ba
bb
bc
bd
be
bf
bg
bh
bi
bj
bk
bl
bm
bn
bo
bp
bs
bt
bu
bv
bw
bx
by
bã
c
c 
c&
ch
c-
c/
c0
c2
c8
c\
ca
cb
cc
cd
ce
cf
cg
ci
cj
ck
cl
cm
cn


In [16]:
# go through each individual infringer and look for the best match of patent company
good_match = 0 
for i in tqdm(range(len(test_match))):
    current_target = test_match.iloc[i]["company_name"]
    first_2letter = current_target[:2]
    # print(first_2letter)

    # if the first two letters matches one of the groups we created ealier, then we proceed to forward
    if first_2letter in all_sort_gp:
        
        candidate_list = unique_patent_co_df.groupby(['sort_gp']).get_group(first_2letter)['patent_company'].tolist()
        # print(len(candidate_list))
        temp = match_name(current_target, candidate_list, 85)
        
        test_match.at[i, "best_match"] = temp[0]
        test_match.at[i, "best_score"] = temp[1]
#         if temp[1] != -1:
#           good_match = good_match+1
#           print(good_match)

100%|██████████| 27693/27693 [37:00<00:00, 12.47it/s]  


In [22]:
test_match = pd.read_csv('public_mapping_2.csv')

In [23]:
test_match.shape

(27693, 4)

In [24]:
unique_match_public = test_match.loc[test_match['best_score'] > 90]

In [25]:
print(unique_match_public.shape)

(928, 4)


In [30]:
#unique_match_public.to_csv('public_company_unique_mapping.csv')  

# now apply the mapping from public company names to patent dataset

In [26]:
# public_copy will now be public_info, because public_info contains all
public_copy = public_info.copy()
public_copy.head(3)

,Unnamed: 0,Company,Year,Currency,Total Assets (millions),Employees (thousands),Company Status,Market Cap (millions),State Incorporated,S&P industry code,...,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,xrd,costat
0,0,01 communique laboratory,2000,CAD,20.226,NaN,A,168.4491,NaN,185.0,...,2000.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,1.561,A
1,1,01 communique laboratory,2001,CAD,7.926,NaN,A,14.4000,NaN,185.0,...,2001.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,1.831,A
2,2,01 communique laboratory,2002,CAD,3.643,NaN,A,5.0954,NaN,185.0,...,2002.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,1.249,A


In [27]:
mapping_domain = unique_match_public.company_name.tolist()

# apply the mapping to the stanford data set
for i in tqdm(range(len(public_copy))):
    old_name = public_copy.iloc[i]["Company"]
    if old_name in mapping_domain: 
        new_name = unique_match_public.loc[unique_match_public['company_name'] == old_name, 'best_match'].iloc[0]
     
        public_copy.at[i, "Company"] = new_name

100%|██████████| 638643/638643 [00:41<00:00, 15332.80it/s]


In [28]:
public_copy

,Unnamed: 0,Company,Year,Currency,Total Assets (millions),Employees (thousands),Company Status,Market Cap (millions),State Incorporated,S&P industry code,...,fyear,indfmt,consol,popsrc,datafmt,tic,conm,curcd,xrd,costat
0,0,01 communique laboratory,2000,CAD,20.226,NaN,A,168.4491,NaN,185.0,...,2000.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,1.561,A
1,1,01 communique laboratory,2001,CAD,7.926,NaN,A,14.4000,NaN,185.0,...,2001.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,1.831,A
2,2,01 communique laboratory,2002,CAD,3.643,NaN,A,5.0954,NaN,185.0,...,2002.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,1.249,A
3,3,01 communique laboratory,2003,CAD,0.874,0.015,A,10.5501,NaN,185.0,...,2003.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,0.439,A
4,4,01 communique laboratory,2004,CAD,1.173,0.023,A,9.6778,NaN,185.0,...,2004.0,INDL,C,D,STD,OONEF,01 communique laboratory,CAD,0.622,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638638,611309,zyqc group holding,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638639,611310,zyqc group holding,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638640,611311,zyqc group holding,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638641,611312,zyqc group holding,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Save the dataset 

In [29]:
public_copy.to_csv('company_size_and_research_fuzzy_matched.csv')  